# Starting the server

In [1]:
%%script bash --bg
python mellow_db/server_run.py

# Using MellowClient

In [2]:
import numpy as np

def normalize_to_unit_length(lst):
    """Normalizes a list to unit length (Euclidean norm of 1) and returns the list."""
    arr = np.array(lst, dtype=np.float64)
    norm = np.linalg.norm(arr)
    return (arr / norm).tolist() if norm != 0 else lst

In [3]:
from mellow_db.client import MellowClient
import json
import os

In [4]:
# Initialize the client
service_account_info = json.loads(os.getenv("GCP_SERVICE_ACCOUNT"))
client = MellowClient("localhost", os.getenv("MELLOW_PORT"), service_account_info)

In [5]:
# Create a demo collection
client.create_collection(
    "demo_collection",
    collection_schema=[
        {"field_name": "name", "field_type": "string", "is_nullable": False},
        {"field_name": "age", "field_type": "integer", "is_nullable": False},
        {"field_name": "details", "field_type": "string", "is_nullable": True},
    ]
)

"Created collection 'demo_collection' successfully"

In [6]:
# Get assigned to the demo collection. After this client will use the same collection for operations
client.use_collection("demo_collection")

"Assigned collection 'demo_collection' successfully"

In [7]:
# Add the first data
client.add(
    key_to_metadata={"123": {"name": "Harry", "age": 25, "details": "Harry is a wizard"}},
    key_to_data={"123": normalize_to_unit_length([0.1, 0.2, 0.3])},
    upsert=False)
client.get_collection_item_count()

1

In [8]:
client.get_collection_info()

{'name': 'demo_collection',
 'item_count': 1,
 'size_in_bytes': 12697,
 'faiss_index_type': 'Flat',
 'faiss_index_metric': 'METRIC_INNER_PRODUCT',
 'embedding_dim': 3,
 'primary_keys': ['key'],
 'meta_columns': [{'name': 'key',
   'type': 'VARCHAR',
   'is_nullable': False,
   'is_index': False,
   'default': None},
  {'name': 'name',
   'type': 'VARCHAR',
   'is_nullable': False,
   'is_index': False,
   'default': None},
  {'name': 'age',
   'type': 'INTEGER',
   'is_nullable': False,
   'is_index': False,
   'default': None},
  {'name': 'details',
   'type': 'VARCHAR',
   'is_nullable': True,
   'is_index': False,
   'default': None}]}

In [9]:
row1 = client.get(
    where={"name": {"$eq": "Harry"}, "age": {"$gte": 20}},
    projection=["name", "details"])
print(f"Row 1:\n{row1}")

# get more fields
row2 = client.get(
    where={"name": {"$eq": "Harry"}, "age": {"$gte": 20}},
    projection=["age", "name", "details", "embedding"])
print(f"Row 2:\n{row2}")

# no rows statisfied the condition
row3 = client.get(
    where={"name": {"$eq": "Harry"}, "age": {"$lte": 20}},
    projection=["age", "name", "details", "embedding"])
print(f"Row 3:\n{row3}")

Row 1:
[('Harry', 'Harry is a wizard')]
Row 2:
[(25, 'Harry', 'Harry is a wizard', [0.2672612419124244, 0.5345224838248488, 0.8017837257372731])]
Row 3:
[]


In [10]:
# update the row
client.add(key_to_metadata={"123": { "age": 37}}, upsert=True)
updated_row = client.get(where={"name": {"$eq": "Harry"}}, projection=["name", "age", "details", "embedding"])
# check how Harry is 37 now
print(f"Updated Row:\n{updated_row}")

Updated Row:
[('Harry', 37, 'Harry is a wizard', [0.2672612419124244, 0.5345224838248488, 0.8017837257372731])]


In [11]:
# add more rows
client.add(key_to_metadata={
    "124": {"name": "Ron", "age": 24, "details": "Ron is Harry's best friend"},
    "125": {"name": "Hermione", "age": 28, "details": "Hermione is smart"},
    "126": {"name": "Dumbledore", "age": 102, "details": "Dumbledore is the best"},
    },
    key_to_data={
        "124": normalize_to_unit_length([0.4, 0.2, 0.3]),
        "125": normalize_to_unit_length([0.7, 0.2, 0.3]),
        "126": normalize_to_unit_length([0.8, 0.2, 0.3]),
    },
    upsert=False
)
# return all rows
client.get(where={}, projection=["key", "name", "age", "details", "embedding"])

[('123',
  'Harry',
  37,
  'Harry is a wizard',
  [0.2672612419124244, 0.5345224838248488, 0.8017837257372731]),
 ('124',
  'Ron',
  24,
  "Ron is Harry's best friend",
  [0.7427813527082073, 0.37139067635410367, 0.5570860145311555]),
 ('125',
  'Hermione',
  28,
  'Hermione is smart',
  [0.8890008890013336, 0.25400025400038106, 0.38100038100057154]),
 ('126',
  'Dumbledore',
  102,
  'Dumbledore is the best',
  [0.9116846116771036, 0.2279211529192759, 0.3418817293789138])]

In [12]:
client.search(
    query_keys=['123'],
    projection=["name", "similarity"],
    n_results=10,
    not_exists_ok=True)

[(('Harry', 1.0),
  ('Ron', 0.8436958193778992),
  ('Hermione', 0.6788442134857178),
  ('Dumbledore', 0.6396021842956543))]

In [13]:
client.search(
    query_keys=['123'],
    projection=["name", "similarity"],
    where={"name": {"$ne": "Hermione"}},
    n_results=10,
    not_exists_ok=True)

[(('Harry', 1.0),
  ('Ron', 0.8436958193778992),
  ('Dumbledore', 0.6396021842956543))]

In [14]:
# can not delete a collection with assigned clients
try:
    delete_response = client.delete_collection("demo_collection")
    print(delete_response)
except Exception as e:
    print(e)

There are 1 active clients connected to collection 'demo_collection'


In [15]:
# disconnect the client when done
client.disconnect()

'Disconnected successfully'

In [16]:
# since disconnected, we can delete the collection
MellowClient(
    "localhost",
    os.getenv("MELLOW_PORT"),
    service_account_info).delete_collection("demo_collection")

"Deleted collection 'demo_collection' successfully"